In [1]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_exif_data(image_path):
    """Extract EXIF data from an image file."""
    exif_data = {}
    image = Image.open(image_path)
    info = image._getexif()
    if info:
        for tag, value in info.items():
            tag_name = TAGS.get(tag, tag)
            if tag_name == "GPSInfo":
                gps_data = {}
                for gps_tag in value:
                    gps_tag_name = GPSTAGS.get(gps_tag, gps_tag)
                    gps_data[gps_tag_name] = value[gps_tag]
                exif_data["GPSInfo"] = gps_data
            else:
                exif_data[tag_name] = value
    return exif_data

def get_coordinates(gps_info):
    """Convert GPSInfo from EXIF to latitude and longitude."""
    def convert_to_degress(value):
        d = value[0][0] / value[0][1]
        m = value[1][0] / value[1][1]
        s = value[2][0] / value[2][1]
        return d + (m / 60.0) + (s / 3600.0)

    lat = convert_to_degress(gps_info['GPSLatitude'])
    if gps_info['GPSLatitudeRef'] != 'N':
        lat = -lat

    lon = convert_to_degress(gps_info['GPSLongitude'])
    if gps_info['GPSLongitudeRef'] != 'E':
        lon = -lon

    return lat, lon

def extract_gps_from_image(image_path):
    """Extract GPS coordinates from an image if available."""
    exif_data = get_exif_data(image_path)
    if "GPSInfo" in exif_data:
        gps_info = exif_data["GPSInfo"]
        lat, lon = get_coordinates(gps_info)
        return lat, lon
    else:
        return None

In [3]:
gps_coordinates = extract_gps_from_image('civic_issue_reporter\media\uploads\broken2_OxEq4jU.jpg')

if gps_coordinates:
    print(f"Latitude: {gps_coordinates[0]}, Longitude: {gps_coordinates[1]}")
else:
    print("No GPS info found in this image.")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 26-27: truncated \uXXXX escape (3354517040.py, line 1)

In [ ]:
lat, lon = 18.5204, 73.8567  # Pune coordinates
address = reverse_geocode(lat, lon)

if address:
    print(f"Detected Address: {address}")
else:
    print("Could not find address.")

In [ ]:
from google.generativeai import GenerativeModel

import datetime

def generate_complaint_with_gemini(description, location):
    today_date = datetime.datetime.now().strftime("%d %B %Y")

    model = GenerativeModel('gemini-1.5-pro')  # or whatever version you use

    prompt = f"""
    You are a civic citizen.
    Write a formal complaint email to the appropriate authority.
    Issue Description: {description}
    Location: {location}
    Date: {today_date}
    Tone: Polite, Professional, Urgent.
    """

    response = model.generate_content(prompt)
    return response.text

# Example
description = "Overflowing garbage near Sector 7 park causing health hazard."
location = "Sector 7 Park, City Center"

complaint_body = generate_complaint_with_gemini(description, location)
print(complaint_body)